In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import fighters_cleanser
import constants

In [3]:
fights = fighters_cleanser.load_cleanse_and_merge(constants.DEFAULT_FIGHTERS_FILE_NAME, constants.DEFAULT_FIGHTS_FILE_NAME)
fights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4393 entries, 0 to 4392
Columns: 136 entries, r_fighter to reach_diff
dtypes: datetime64[ns](3), float64(21), int64(81), object(31)
memory usage: 4.6+ MB


In [4]:
fights.r_b_winner.value_counts() / fights.shape[0]

r       0.630776
b       0.353062
None    0.016162
Name: r_b_winner, dtype: float64

In [5]:
fights.head()

,r_fighter,b_fighter,r_kd,b_kd,r_sig_str,b_sig_str,r_sig_str_pct,b_sig_str_pct,r_total_str,b_total_str,...,b_weight,b_reach,b_stance,b_dob,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff
0,Henry Cejudo,Marlon Moraes,0,0,90 of 171,57 of 119,52,47,99 of 182,59 of 121,...,135,67,Orthodox,1988-04-26,32.347945,31.136986,1.210959,-2,0,-3
1,Jimmie Rivera,Marlon Moraes,0,1,0 of 3,7 of 9,0,77,0 of 3,7 of 9,...,135,67,Orthodox,1988-04-26,28.942466,30.117808,-1.175342,-2,0,1
2,John Dodson,Marlon Moraes,1,0,43 of 105,45 of 131,40,34,47 of 109,45 of 131,...,135,67,Orthodox,1988-04-26,33.147945,29.564384,3.583562,-3,0,-1
3,Raphael Assuncao,Marlon Moraes,0,1,2 of 12,10 of 23,16,43,3 of 13,12 of 25,...,135,67,Orthodox,1988-04-26,36.567123,30.791781,5.775342,-1,0,-1
4,Raphael Assuncao,Marlon Moraes,0,0,43 of 134,44 of 150,32,29,43 of 134,44 of 150,...,135,67,Orthodox,1988-04-26,34.898630,29.123288,5.775342,-1,0,-1


In [6]:
#list(fights.columns)

In [7]:
#I would like to to remove a bunch of colums.  All the ones we made for the other analysis are not relevant, since this time we are looking only at stats from before the fight.

In [8]:
relevant_columns = ['r_fighter', 'b_fighter', 'winner', 'loser', 'r_height', 'r_weight','r_reach','r_stance', 'b_height','b_weight','b_reach','b_stance','r_age','b_age',
                    'age_diff', 'height_diff', 'weight_diff', 'reach_diff', 'r_b_winner']
fights = fights[relevant_columns]

In [9]:
fights.head()

,r_fighter,b_fighter,winner,loser,r_height,r_weight,r_reach,r_stance,b_height,b_weight,b_reach,b_stance,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_b_winner
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,Marlon Moraes,64,135,64,Orthodox,66,135,67,Orthodox,32.347945,31.136986,1.210959,-2,0,-3,r
1,Jimmie Rivera,Marlon Moraes,Marlon Moraes,Jimmie Rivera,64,135,68,Orthodox,66,135,67,Orthodox,28.942466,30.117808,-1.175342,-2,0,1,b
2,John Dodson,Marlon Moraes,Marlon Moraes,John Dodson,63,135,66,Orthodox,66,135,67,Orthodox,33.147945,29.564384,3.583562,-3,0,-1,b
3,Raphael Assuncao,Marlon Moraes,Marlon Moraes,Raphael Assuncao,65,135,66,Orthodox,66,135,67,Orthodox,36.567123,30.791781,5.775342,-1,0,-1,b
4,Raphael Assuncao,Marlon Moraes,Raphael Assuncao,Marlon Moraes,65,135,66,Orthodox,66,135,67,Orthodox,34.898630,29.123288,5.775342,-1,0,-1,r


In [10]:
# Do diff columns for weight, height, and reach.
# I suspect I want to keep the raw age, reach, etc. because it might potentially be relevant.
# I suspect age and reach will be much more relevant than height and weight.  Weight should be fairly close (except in the earlier ones), and reach is probably more important than height.  Well, height might tell you something about kicking reach.

In [11]:
# not sure if stance is relevant.  If we do DNN, we'll need to convert it to categorical.
# I think that for both logistic regression and DNN, we'll need to scale the numeric variables as we did before.
# What other potential algorithms can we use?
# Random Forests?  Naive Bayes?

In [12]:
# How important is it to take into account the previous record, and how?  Would we use the same alg, or a different one?  Will something besides DNN work?

In [13]:
fights.shape

(4393, 19)

In [14]:
# Is that enough data points to train something?

In [15]:
#fights.r_stance.value_counts()

In [16]:
#fights.b_stance.value_counts()

Next:

scale the numeric variables
add an r_won variable that is 0 or 1 as in the other DNN analysis
"categorize" the stance variables - will result in 4 extra variable for each since there are 4 values, each being 0 or 1

In [17]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [18]:
def get_numeric_stance(stance):
    if stance == 'Orthodox':
        return 0
    elif stance == 'Southpaw':
        return 1
    elif stance == 'Switch':
        return 2
    elif stance == 'Open Stance':
        return 3
    else:
        return 4

In [19]:
r_stance_numeric = fights.r_stance.apply(get_numeric_stance)

In [20]:
#r_stance_numeric.value_counts()

In [21]:
#fights.r_stance.value_counts()

In [22]:
b_stance_numeric = fights.b_stance.apply(get_numeric_stance)
#b_stance_numeric.value_counts()

In [23]:
#fights.b_stance.value_counts()

In [24]:
fights.r_stance.isnull().sum()

119

In [25]:
fights.b_stance.isnull().sum()

116

There are 119 fights where red stance is missing and 116 where blue stance is missing.

I don't want to discard all fights where there is no value for stance.  I'll either not count it, or just accept those unknown values as "4".  Sigh.

In [26]:
rsc = to_categorical(r_stance_numeric)
rsc.shape

(4393, 5)

In [27]:
bsc = to_categorical(b_stance_numeric)
bsc.shape

(4393, 5)

In [28]:
fights.shape

(4393, 19)

In [29]:
fights.head()

,r_fighter,b_fighter,winner,loser,r_height,r_weight,r_reach,r_stance,b_height,b_weight,b_reach,b_stance,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_b_winner
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,Marlon Moraes,64,135,64,Orthodox,66,135,67,Orthodox,32.347945,31.136986,1.210959,-2,0,-3,r
1,Jimmie Rivera,Marlon Moraes,Marlon Moraes,Jimmie Rivera,64,135,68,Orthodox,66,135,67,Orthodox,28.942466,30.117808,-1.175342,-2,0,1,b
2,John Dodson,Marlon Moraes,Marlon Moraes,John Dodson,63,135,66,Orthodox,66,135,67,Orthodox,33.147945,29.564384,3.583562,-3,0,-1,b
3,Raphael Assuncao,Marlon Moraes,Marlon Moraes,Raphael Assuncao,65,135,66,Orthodox,66,135,67,Orthodox,36.567123,30.791781,5.775342,-1,0,-1,b
4,Raphael Assuncao,Marlon Moraes,Raphael Assuncao,Marlon Moraes,65,135,66,Orthodox,66,135,67,Orthodox,34.898630,29.123288,5.775342,-1,0,-1,r


In [30]:
fights_backup = fights.copy()
fights_backup.head()

,r_fighter,b_fighter,winner,loser,r_height,r_weight,r_reach,r_stance,b_height,b_weight,b_reach,b_stance,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_b_winner
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,Marlon Moraes,64,135,64,Orthodox,66,135,67,Orthodox,32.347945,31.136986,1.210959,-2,0,-3,r
1,Jimmie Rivera,Marlon Moraes,Marlon Moraes,Jimmie Rivera,64,135,68,Orthodox,66,135,67,Orthodox,28.942466,30.117808,-1.175342,-2,0,1,b
2,John Dodson,Marlon Moraes,Marlon Moraes,John Dodson,63,135,66,Orthodox,66,135,67,Orthodox,33.147945,29.564384,3.583562,-3,0,-1,b
3,Raphael Assuncao,Marlon Moraes,Marlon Moraes,Raphael Assuncao,65,135,66,Orthodox,66,135,67,Orthodox,36.567123,30.791781,5.775342,-1,0,-1,b
4,Raphael Assuncao,Marlon Moraes,Raphael Assuncao,Marlon Moraes,65,135,66,Orthodox,66,135,67,Orthodox,34.898630,29.123288,5.775342,-1,0,-1,r


In [31]:
scaled_fights = fighters_cleanser.scale_columns(fights, ['r_height', 'r_weight', 'r_reach', 'b_height', 'b_weight', 'b_reach', 'age_diff', 'height_diff', 'weight_diff', 'reach_diff'])
scaled_fights.head()

,r_fighter,b_fighter,winner,loser,r_height,r_weight,r_reach,r_stance,b_height,b_weight,b_reach,b_stance,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_b_winner
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,Marlon Moraes,-1.863237,-1.006467,-1.999412,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,32.347945,31.136986,0.146347,-0.780406,-0.054475,-0.924498,r
1,Jimmie Rivera,Marlon Moraes,Marlon Moraes,Jimmie Rivera,-1.863237,-1.006467,-1.025344,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,28.942466,30.117808,-0.318509,-0.780406,-0.054475,0.286752,b
2,John Dodson,Marlon Moraes,Marlon Moraes,John Dodson,-2.154930,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,33.147945,29.564384,0.608535,-1.176881,-0.054475,-0.318873,b
3,Raphael Assuncao,Marlon Moraes,Marlon Moraes,Raphael Assuncao,-1.571544,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,36.567123,30.791781,1.035498,-0.383930,-0.054475,-0.318873,b
4,Raphael Assuncao,Marlon Moraes,Raphael Assuncao,Marlon Moraes,-1.571544,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,34.898630,29.123288,1.035498,-0.383930,-0.054475,-0.318873,r


In [32]:
fights.head()

,r_fighter,b_fighter,winner,loser,r_height,r_weight,r_reach,r_stance,b_height,b_weight,b_reach,b_stance,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_b_winner
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,Marlon Moraes,-1.863237,-1.006467,-1.999412,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,32.347945,31.136986,0.146347,-0.780406,-0.054475,-0.924498,r
1,Jimmie Rivera,Marlon Moraes,Marlon Moraes,Jimmie Rivera,-1.863237,-1.006467,-1.025344,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,28.942466,30.117808,-0.318509,-0.780406,-0.054475,0.286752,b
2,John Dodson,Marlon Moraes,Marlon Moraes,John Dodson,-2.154930,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,33.147945,29.564384,0.608535,-1.176881,-0.054475,-0.318873,b
3,Raphael Assuncao,Marlon Moraes,Marlon Moraes,Raphael Assuncao,-1.571544,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,36.567123,30.791781,1.035498,-0.383930,-0.054475,-0.318873,b
4,Raphael Assuncao,Marlon Moraes,Raphael Assuncao,Marlon Moraes,-1.571544,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,-1.288803,Orthodox,34.898630,29.123288,1.035498,-0.383930,-0.054475,-0.318873,r


In [33]:
fights_backup.head()

,r_fighter,b_fighter,winner,loser,r_height,r_weight,r_reach,r_stance,b_height,b_weight,b_reach,b_stance,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_b_winner
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,Marlon Moraes,64,135,64,Orthodox,66,135,67,Orthodox,32.347945,31.136986,1.210959,-2,0,-3,r
1,Jimmie Rivera,Marlon Moraes,Marlon Moraes,Jimmie Rivera,64,135,68,Orthodox,66,135,67,Orthodox,28.942466,30.117808,-1.175342,-2,0,1,b
2,John Dodson,Marlon Moraes,Marlon Moraes,John Dodson,63,135,66,Orthodox,66,135,67,Orthodox,33.147945,29.564384,3.583562,-3,0,-1,b
3,Raphael Assuncao,Marlon Moraes,Marlon Moraes,Raphael Assuncao,65,135,66,Orthodox,66,135,67,Orthodox,36.567123,30.791781,5.775342,-1,0,-1,b
4,Raphael Assuncao,Marlon Moraes,Raphael Assuncao,Marlon Moraes,65,135,66,Orthodox,66,135,67,Orthodox,34.898630,29.123288,5.775342,-1,0,-1,r


Calling fighters_cleanser.scale_columns() on fights will change fights.  It doesn't just return a modified DataFrame.  Is that a defect in my code, or something with Pandas.

I suspect it is from this line in scale_column():  "column = column.apply(lambda n: (n - mean) / std)"

In [34]:
rsc

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [35]:
rsc.T

array([[1., 1., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [36]:
scaled_fights['r_orthodox'] = rsc.T[0]
scaled_fights['r_southpaw'] = rsc.T[1]
scaled_fights['r_switch'] = rsc.T[2]
scaled_fights['r_open'] = rsc.T[3]

In [37]:
scaled_fights['b_orthodox'] = bsc.T[0]
scaled_fights['b_southpaw'] = bsc.T[1]
scaled_fights['b_switch'] = bsc.T[2]
scaled_fights['b_open'] = bsc.T[3]

In [38]:
scaled_fights['r_won'] = scaled_fights.r_b_winner.apply(lambda x: 1 if x == 'r' else 0)

In [39]:
#scaled_fights = scaled_fights.drop(columns=['r_stance', 'b_stance'])

In [40]:
#scaled_fights = scaled_fights.drop(columns=['r_fighter', 'b_fighter', 'winner', 'loser', 'r_b_winner'])

In [41]:
scaled_fights.head(29)

,r_fighter,b_fighter,winner,loser,r_height,r_weight,r_reach,r_stance,b_height,b_weight,...,r_b_winner,r_orthodox,r_southpaw,r_switch,r_open,b_orthodox,b_southpaw,b_switch,b_open,r_won
0,Henry Cejudo,Marlon Moraes,Henry Cejudo,Marlon Moraes,-1.863237,-1.006467,-1.999412,Orthodox,-1.313331,-0.995861,...,r,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
1,Jimmie Rivera,Marlon Moraes,Marlon Moraes,Jimmie Rivera,-1.863237,-1.006467,-1.025344,Orthodox,-1.313331,-0.995861,...,b,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,John Dodson,Marlon Moraes,Marlon Moraes,John Dodson,-2.154930,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,...,b,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,Raphael Assuncao,Marlon Moraes,Marlon Moraes,Raphael Assuncao,-1.571544,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,...,b,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,Raphael Assuncao,Marlon Moraes,Raphael Assuncao,Marlon Moraes,-1.571544,-1.006467,-1.512378,Orthodox,-1.313331,-0.995861,...,r,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
5,Henry Cejudo,TJ Dillashaw,Henry Cejudo,TJ Dillashaw,-1.863237,-1.006467,-1.999412,Orthodox,-1.313331,-1.296560,...,r,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
6,Cody Garbrandt,TJ Dillashaw,TJ Dillashaw,Cody Garbrandt,-0.696465,-1.006467,-1.755895,Orthodox,-1.313331,-1.296560,...,b,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
7,Renan Barao,TJ Dillashaw,TJ Dillashaw,Renan Barao,-1.279851,-1.006467,-0.538310,Orthodox,-1.313331,-1.296560,...,b,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
8,Raphael Assuncao,TJ Dillashaw,Raphael Assuncao,TJ Dillashaw,-1.571544,-1.006467,-1.512378,Orthodox,-1.313331,-1.296560,...,r,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
9,Henry Cejudo,Sergio Pettis,Henry Cejudo,Sergio Pettis,-1.863237,-1.006467,-1.999412,Orthodox,-1.313331,-0.995861,...,r,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1


Now I think we are ready to put it into logistic regression or nn

In [42]:
scaled_fights.columns

Index(['r_fighter', 'b_fighter', 'winner', 'loser', 'r_height', 'r_weight',
       'r_reach', 'r_stance', 'b_height', 'b_weight', 'b_reach', 'b_stance',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff', 'r_b_winner', 'r_orthodox', 'r_southpaw', 'r_switch',
       'r_open', 'b_orthodox', 'b_southpaw', 'b_switch', 'b_open', 'r_won'],
      dtype='object')

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [44]:
x_train, x_test, y_train, y_test = train_test_split(scaled_fights[['r_height', 'r_weight', 'r_reach', 'b_height', 'b_weight', 'b_reach',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff', 'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], scaled_fights['r_won'], test_size=.33, random_state=1)

In [45]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2943, 20), (1450, 20), (2943,), (1450,))

In [46]:
x_train.head()

,r_height,r_weight,r_reach,b_height,b_weight,b_reach,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_orthodox,r_southpaw,r_switch,r_open,b_orthodox,b_southpaw,b_switch,b_open
521,-0.696465,0.038346,-0.538310,-0.421790,-0.394462,-0.539139,24.915068,28.158904,-0.721455,-0.383930,1.313521,-0.016061,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2336,-0.113078,0.038346,-0.051276,-0.421790,-0.394462,-0.289251,34.871233,24.816438,1.869143,0.409020,1.313521,0.286752,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3579,-0.113078,0.038346,-0.051276,-0.421790,0.056587,-0.289251,27.545205,32.098630,-0.976566,0.409020,-0.054475,0.286752,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4184,-0.113078,0.038346,0.192242,0.469752,0.056587,0.210525,28.731507,27.115068,0.225335,-0.780406,-0.054475,-0.016061,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
600,0.178615,-0.707949,0.435759,-0.124609,-0.695161,0.210525,28.668493,30.317808,-0.410840,0.409020,-0.054475,0.286752,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [47]:
x_test.head()

,r_height,r_weight,r_reach,b_height,b_weight,b_reach,r_age,b_age,age_diff,height_diff,weight_diff,reach_diff,r_orthodox,r_southpaw,r_switch,r_open,b_orthodox,b_southpaw,b_switch,b_open
343,-0.113078,0.038346,0.435759,1.064114,1.109036,1.459965,28.397260,24.386301,0.691793,-1.573356,-3.246465,-1.227310,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2325,1.345387,1.083159,1.166310,0.766933,1.109036,0.710301,28.504110,30.989041,-0.573619,0.805496,-0.054475,0.589564,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3035,0.470308,0.038346,0.192242,0.172572,0.056587,-0.039363,28.769863,28.832877,-0.101825,0.409020,-0.054475,0.286752,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2785,0.470308,0.486123,1.653344,0.766933,0.507637,-0.039363,33.956164,27.230137,1.220693,-0.383930,-0.054475,2.103626,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133,0.178615,0.038346,0.435759,-0.718970,-0.394462,-0.789027,20.153425,21.430137,-0.338256,1.201971,1.313521,1.498001,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [48]:
y_train.head()

521     1
2336    1
3579    1
4184    1
600     1
Name: r_won, dtype: int64

In [49]:
y_test.head()

343     1
2325    1
3035    0
2785    0
133     1
Name: r_won, dtype: int64

In [50]:
lr = LogisticRegression()

In [51]:
lr.fit(x_train, y_train)

C:\Users\paulj_1e1uzlz\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [52]:
predictions = lr.predict(x_test)

In [53]:
confusion_matrix(y_test, predictions)

array([[ 91, 475],
       [ 71, 813]], dtype=int64)

In [54]:
accuracy_score(y_test, predictions)

0.623448275862069

In [55]:
pd.Series(predictions).value_counts() / len(predictions)

1    0.888276
0    0.111724
dtype: float64

In [56]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.56      0.16      0.25       566
           1       0.63      0.92      0.75       884

    accuracy                           0.62      1450
   macro avg       0.60      0.54      0.50      1450
weighted avg       0.60      0.62      0.55      1450



In [57]:
scaled_fights.r_won.sum() / scaled_fights.shape[0]

0.6307762349191897

Logistic regression was about 62% accurate, but guession "R" every time gets you 63%.  So logistic regression didn't do all that well.

One option is to try it on fewer variables.  Drop all the stance variables and see if that makes a difference.

In [58]:
scaled_fights.columns

Index(['r_fighter', 'b_fighter', 'winner', 'loser', 'r_height', 'r_weight',
       'r_reach', 'r_stance', 'b_height', 'b_weight', 'b_reach', 'b_stance',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff', 'r_b_winner', 'r_orthodox', 'r_southpaw', 'r_switch',
       'r_open', 'b_orthodox', 'b_southpaw', 'b_switch', 'b_open', 'r_won'],
      dtype='object')

In [59]:
#scaled_fights = scaled_fights.drop(columns=['r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
#       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open'])
# You don't need to drop columns.  You can specificy which columsn you use in train_test_split

In [60]:
x_train, x_test, y_train, y_test = train_test_split(scaled_fights[['r_height', 'r_weight', 'r_reach', 'b_height', 'b_weight', 'b_reach',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff']], scaled_fights['r_won'], test_size=.33, random_state=1)

In [61]:
lr2 = LogisticRegression()

In [62]:
lr2.fit(x_train, y_train)

C:\Users\paulj_1e1uzlz\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [63]:
predictions2 = lr2.predict(x_test)

In [64]:
accuracy_score(y_test, predictions2)

0.6172413793103448

In [65]:
pd.Series(predictions2).value_counts() / len(predictions2)

1    0.895862
0    0.104138
dtype: float64

In [66]:
confusion_matrix(y_test, predictions2)

array([[ 81, 485],
       [ 70, 814]], dtype=int64)

In [67]:
print(classification_report(y_test, predictions2))

              precision    recall  f1-score   support

           0       0.54      0.14      0.23       566
           1       0.63      0.92      0.75       884

    accuracy                           0.62      1450
   macro avg       0.58      0.53      0.49      1450
weighted avg       0.59      0.62      0.54      1450



So even less accurate.  One last thing before going to neural networks.  Try stances only.

In [68]:
x_train, x_test, y_train, y_test = train_test_split(scaled_fights[['r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], scaled_fights['r_won'], test_size=.33, random_state=1)

In [69]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2943, 8), (1450, 8), (2943,), (1450,))

In [70]:
x_train.head()

,r_orthodox,r_southpaw,r_switch,r_open,b_orthodox,b_southpaw,b_switch,b_open
521,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2336,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3579,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4184,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
600,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [71]:
lr3 = LogisticRegression()
lr3.fit(x_train, y_train)

LogisticRegression()

In [72]:
predictions3 = lr3.predict(x_test)

In [73]:
accuracy_score(y_test, predictions3)

0.6096551724137931

In [74]:
pd.Series(predictions3).value_counts() / len(predictions3)

1    1.0
dtype: float64

In [75]:
confusion_matrix(y_test, predictions3)

array([[  0, 566],
       [  0, 884]], dtype=int64)

In [76]:
print(classification_report(y_test, predictions3))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       566
           1       0.61      1.00      0.76       884

    accuracy                           0.61      1450
   macro avg       0.30      0.50      0.38      1450
weighted avg       0.37      0.61      0.46      1450



C:\Users\paulj_1e1uzlz\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


We keep getting worse and worse, although only be small amounts.

Time to try DNN and see what we can do.

------------------------------------
DNN Section

In [77]:
from keras.models import Sequential
from keras.layers import Dense

In [78]:
x_train, x_test, y_train, y_test = train_test_split(scaled_fights[['r_height', 'r_weight', 'r_reach', 'b_height', 'b_weight', 'b_reach',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff', 'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], scaled_fights['r_won'], test_size=.33, random_state=1)

In [79]:
y_binary = to_categorical(y_train)

In [80]:
model = Sequential()
model.add(Dense(9, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(9, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [81]:
model.fit(x_train, y_binary, epochs=100, verbose=2)

Epoch 1/100
 - 1s - loss: 1.5352 - accuracy: 0.4774
Epoch 2/100
 - 0s - loss: 0.6639 - accuracy: 0.6330
Epoch 3/100
 - 0s - loss: 0.6494 - accuracy: 0.6405
Epoch 4/100
 - 1s - loss: 0.6418 - accuracy: 0.6446
Epoch 5/100
 - 0s - loss: 0.6382 - accuracy: 0.6429
Epoch 6/100
 - 0s - loss: 0.6374 - accuracy: 0.6429
Epoch 7/100
 - 0s - loss: 0.6350 - accuracy: 0.6391
Epoch 8/100
 - 0s - loss: 0.6342 - accuracy: 0.6412
Epoch 9/100
 - 0s - loss: 0.6377 - accuracy: 0.6412
Epoch 10/100
 - 0s - loss: 0.6345 - accuracy: 0.6487
Epoch 11/100
 - 0s - loss: 0.6335 - accuracy: 0.6476
Epoch 12/100
 - 0s - loss: 0.6339 - accuracy: 0.6436
Epoch 13/100
 - 0s - loss: 0.6333 - accuracy: 0.6449
Epoch 14/100
 - 0s - loss: 0.6319 - accuracy: 0.6449
Epoch 15/100
 - 0s - loss: 0.6329 - accuracy: 0.6490
Epoch 16/100
 - 0s - loss: 0.6327 - accuracy: 0.6524
Epoch 17/100
 - 0s - loss: 0.6310 - accuracy: 0.6504
Epoch 18/100
 - 0s - loss: 0.6311 - accuracy: 0.6419
Epoch 19/100
 - 0s - loss: 0.6314 - accuracy: 0.6470
Ep

In [82]:
dnn_predictions = model.predict(x_test)

In [83]:
dnn_predictions.shape

(1450, 2)

In [84]:
x_test.shape

(1450, 20)

In [85]:
y_test_binary = to_categorical(y_test)

In [86]:
pred = np.apply_along_axis(lambda r: 1 if r[1] > r[0] else 0, 1, dnn_predictions)

In [87]:
accuracy_score(y_test, pred)

0.6062068965517241

In [88]:
confusion_matrix(y_test, pred)

array([[187, 379],
       [192, 692]], dtype=int64)

In [89]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.49      0.33      0.40       566
           1       0.65      0.78      0.71       884

    accuracy                           0.61      1450
   macro avg       0.57      0.56      0.55      1450
weighted avg       0.59      0.61      0.59      1450



This is not all that accurate either.  One possibility is...you simply can't predict the winner very well one just stance and body type.

But it is better than 50/50, so there must be some sort of information in there.  I am curious which columns give you that.  One more thing to try would be adding back the red/blue information and see if we can get significantly better than the baseline of "red always wins" for ~67% accuracy.

In [90]:
fights.r_won.sum() / fights.shape[0]

0.6307762349191897

In [91]:
fights.r_b_winner.value_counts()

r       2771
b       1551
None      71
Name: r_b_winner, dtype: int64

In [92]:
fights.r_b_winner.value_counts() / fights.shape[0]

r       0.630776
b       0.353062
None    0.016162
Name: r_b_winner, dtype: float64

OK, we need to account for this...There are some without winners.  For this ML algorithm, we'll need to removed those if we are predicting a binary red/blue.  If we want to keep them, we'll need to be able to predict between three outcomes.

It is interesting that the accuracy here is very close to the "red always wins" strategy.  I wonder if that is coincidence or not.

In [93]:
pd.Series(pred).value_counts()

1    1071
0     379
dtype: int64

In [94]:
pd.Series(pred).value_counts() / pred.shape[0]

1    0.738621
0    0.261379
dtype: float64

So it guessed "red won"for about 93 percent of them.

In [95]:
pd.Series(y_test).value_counts()

1    884
0    566
Name: r_won, dtype: int64

In [96]:
pd.Series(y_test).value_counts() / y_test.shape[0]

1    0.609655
0    0.390345
Name: r_won, dtype: float64

In [97]:
a = np.correlate(y_test, pred)
a

array([692], dtype=int64)

In [98]:
a[0]

692

In [99]:
np.corrcoef(y_test, pred)

array([[1.        , 0.12567355],
       [0.12567355, 1.        ]])

In [100]:
np.cov(y_test, pred)

array([[0.23813998, 0.02695605],
       [0.02695605, 0.1931934 ]])

In [101]:
np.cov([1, 2, 3, 4], [1, 2, 3, 4])

array([[1.66666667, 1.66666667],
       [1.66666667, 1.66666667]])

In [102]:
np.corrcoef([1, 2, 3, 4], [1, 2, 3, 4])

array([[1., 1.],
       [1., 1.]])

In [103]:
np.correlate([1, 2, 3, 4], [1, 2, 3, 4])

array([30])

In [104]:
confusion_matrix(y_test, pred)

array([[187, 379],
       [192, 692]], dtype=int64)

In [105]:
y_test.head()

343     1
2325    1
3035    0
2785    0
133     1
Name: r_won, dtype: int64

In [106]:
pred[:5]

array([1, 1, 1, 1, 1])

In [107]:
(y_test + pred).value_counts()

2    692
1    571
0    187
Name: r_won, dtype: int64

In [108]:
(y_test * -1 + 2 * pred).value_counts()

 1    692
 2    379
-1    192
 0    187
Name: r_won, dtype: int64

In [109]:
# 63 - when it was yes but it guessed no (wrong)
# 75 - when it was no and it guessed no (correct)
# 491 - when it was no but it guessed yes (wrong)
# 821 - when it was yes and it guessed yes (correct)

In [110]:
# It guessed "red won" for 93%, and got a lot of false positives.  Why is it so skewed toward the "red" side?

In [111]:
from DNN_analyzer import DNNAnalyzer
dnn = DNNAnalyzer()

In [112]:
#dnn.analyze_5(x_columns=fights[['r_height', 'r_weight', 'r_reach', 'b_height', 'b_weight', 'b_reach',
#       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
#       'reach_diff', 'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
#       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], y_column=fights.r_won)

In [113]:
dnn.run_analysis(model, x_columns=fights[['r_height', 'r_weight', 'r_reach', 'b_height', 'b_weight', 'b_reach',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff', 'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], y_column=fights.r_won)

Epoch 1/100
 - 0s - loss: 0.6208 - accuracy: 0.6585
Epoch 2/100
 - 0s - loss: 0.6198 - accuracy: 0.6595
Epoch 3/100
 - 0s - loss: 0.6195 - accuracy: 0.6490
Epoch 4/100
 - 0s - loss: 0.6203 - accuracy: 0.6616
Epoch 5/100
 - 0s - loss: 0.6202 - accuracy: 0.6592
Epoch 6/100
 - 0s - loss: 0.6182 - accuracy: 0.6572
Epoch 7/100
 - 0s - loss: 0.6224 - accuracy: 0.6521
Epoch 8/100
 - 0s - loss: 0.6199 - accuracy: 0.6609
Epoch 9/100
 - 0s - loss: 0.6198 - accuracy: 0.6585
Epoch 10/100
 - 0s - loss: 0.6186 - accuracy: 0.6565
Epoch 11/100
 - 0s - loss: 0.6185 - accuracy: 0.6561
Epoch 12/100
 - 0s - loss: 0.6196 - accuracy: 0.6589
Epoch 13/100
 - 0s - loss: 0.6193 - accuracy: 0.6575
Epoch 14/100
 - 0s - loss: 0.6201 - accuracy: 0.6561
Epoch 15/100
 - 0s - loss: 0.6179 - accuracy: 0.6572
Epoch 16/100
 - 0s - loss: 0.6178 - accuracy: 0.6653
Epoch 17/100
 - 0s - loss: 0.6169 - accuracy: 0.6606
Epoch 18/100
 - 0s - loss: 0.6180 - accuracy: 0.6578
Epoch 19/100
 - 0s - loss: 0.6190 - accuracy: 0.6595
Ep

In [114]:
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [115]:
dnn.run_analysis(model, x_columns=fights[['r_height', 'r_weight', 'r_reach', 'b_height', 'b_weight', 'b_reach',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff']], y_column=fights.r_won)

Epoch 1/100
 - 0s - loss: 1.1485 - accuracy: 0.6035
Epoch 2/100
 - 0s - loss: 0.6633 - accuracy: 0.6211
Epoch 3/100
 - 0s - loss: 0.6563 - accuracy: 0.6259
Epoch 4/100
 - 0s - loss: 0.6478 - accuracy: 0.6317
Epoch 5/100
 - 0s - loss: 0.6460 - accuracy: 0.6351
Epoch 6/100
 - 0s - loss: 0.6462 - accuracy: 0.6334
Epoch 7/100
 - 0s - loss: 0.6443 - accuracy: 0.6344
Epoch 8/100
 - 0s - loss: 0.6446 - accuracy: 0.6374
Epoch 9/100
 - 0s - loss: 0.6407 - accuracy: 0.6388
Epoch 10/100
 - 0s - loss: 0.6403 - accuracy: 0.6425
Epoch 11/100
 - 0s - loss: 0.6411 - accuracy: 0.6388
Epoch 12/100
 - 0s - loss: 0.6496 - accuracy: 0.6381
Epoch 13/100
 - 0s - loss: 0.6403 - accuracy: 0.6374
Epoch 14/100
 - 0s - loss: 0.6426 - accuracy: 0.6385
Epoch 15/100
 - 0s - loss: 0.6403 - accuracy: 0.6402
Epoch 16/100
 - 0s - loss: 0.6400 - accuracy: 0.6459
Epoch 17/100
 - 0s - loss: 0.6391 - accuracy: 0.6436
Epoch 18/100
 - 0s - loss: 0.6375 - accuracy: 0.6446
Epoch 19/100
 - 0s - loss: 0.6382 - accuracy: 0.6517
Ep

In [116]:
model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [117]:
dnn.run_analysis(model, x_columns=fights[['r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], y_column=fights.r_won)

Epoch 1/100
 - 0s - loss: 0.6630 - accuracy: 0.6082
Epoch 2/100
 - 0s - loss: 0.6521 - accuracy: 0.6415
Epoch 3/100
 - 0s - loss: 0.6512 - accuracy: 0.6415
Epoch 4/100
 - 0s - loss: 0.6506 - accuracy: 0.6415
Epoch 5/100
 - 1s - loss: 0.6504 - accuracy: 0.6415
Epoch 6/100
 - 1s - loss: 0.6505 - accuracy: 0.6415
Epoch 7/100
 - 0s - loss: 0.6500 - accuracy: 0.6415
Epoch 8/100
 - 0s - loss: 0.6502 - accuracy: 0.6415
Epoch 9/100
 - 0s - loss: 0.6499 - accuracy: 0.6415
Epoch 10/100
 - 0s - loss: 0.6498 - accuracy: 0.6415
Epoch 11/100
 - 0s - loss: 0.6501 - accuracy: 0.6415
Epoch 12/100
 - 0s - loss: 0.6500 - accuracy: 0.6415
Epoch 13/100
 - 0s - loss: 0.6501 - accuracy: 0.6415
Epoch 14/100
 - 0s - loss: 0.6497 - accuracy: 0.6415
Epoch 15/100
 - 0s - loss: 0.6498 - accuracy: 0.6415
Epoch 16/100
 - 0s - loss: 0.6499 - accuracy: 0.6415
Epoch 17/100
 - 0s - loss: 0.6497 - accuracy: 0.6415
Epoch 18/100
 - 0s - loss: 0.6498 - accuracy: 0.6415
Epoch 19/100
 - 0s - loss: 0.6495 - accuracy: 0.6415
Ep

In [118]:
import importlib

In [119]:
from importlib import reload

In [122]:
from DNN_analyzer import DNNAnalyzer
dnn = DNNAnalyzer()

In [123]:
dnn.run_analysis(model, x_columns=fights[['r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], y_column=fights.r_won, num_epochs=40)

Epoch 1/40
 - 1s - loss: 0.6489 - accuracy: 0.6412
Epoch 2/40
 - 1s - loss: 0.6492 - accuracy: 0.6412
Epoch 3/40
 - 1s - loss: 0.6486 - accuracy: 0.6419
Epoch 4/40
 - 1s - loss: 0.6492 - accuracy: 0.6408
Epoch 5/40
 - 1s - loss: 0.6489 - accuracy: 0.6415
Epoch 6/40
 - 1s - loss: 0.6489 - accuracy: 0.6419
Epoch 7/40
 - 1s - loss: 0.6488 - accuracy: 0.6415
Epoch 8/40
 - 1s - loss: 0.6488 - accuracy: 0.6419
Epoch 9/40
 - 0s - loss: 0.6489 - accuracy: 0.6415
Epoch 10/40
 - 0s - loss: 0.6491 - accuracy: 0.6408
Epoch 11/40
 - 1s - loss: 0.6488 - accuracy: 0.6405
Epoch 12/40
 - 0s - loss: 0.6488 - accuracy: 0.6415
Epoch 13/40
 - 0s - loss: 0.6492 - accuracy: 0.6415
Epoch 14/40
 - 0s - loss: 0.6491 - accuracy: 0.6415
Epoch 15/40
 - 0s - loss: 0.6489 - accuracy: 0.6412
Epoch 16/40
 - 0s - loss: 0.6492 - accuracy: 0.6408
Epoch 17/40
 - 0s - loss: 0.6488 - accuracy: 0.6415
Epoch 18/40
 - 1s - loss: 0.6491 - accuracy: 0.6415
Epoch 19/40
 - 1s - loss: 0.6493 - accuracy: 0.6412
Epoch 20/40
 - 1s - l

In [124]:
pd.Series(y_test).value_counts() / len(y_test)

1    0.609655
0    0.390345
Name: r_won, dtype: float64

In [125]:
scaled_fights.r_won.value_counts() / scaled_fights.shape[0]

1    0.630776
0    0.369224
Name: r_won, dtype: float64

In [126]:
dnn.run_analysis(model, x_columns=scaled_fights[['r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']], y_column=fights.r_won, num_epochs=15)

Epoch 1/15
 - 0s - loss: 0.6487 - accuracy: 0.6415
Epoch 2/15
 - 0s - loss: 0.6489 - accuracy: 0.6415
Epoch 3/15
 - 0s - loss: 0.6486 - accuracy: 0.6419
Epoch 4/15
 - 0s - loss: 0.6488 - accuracy: 0.6415
Epoch 5/15
 - 0s - loss: 0.6490 - accuracy: 0.6415
Epoch 6/15
 - 0s - loss: 0.6487 - accuracy: 0.6408
Epoch 7/15
 - 0s - loss: 0.6488 - accuracy: 0.6408
Epoch 8/15
 - 0s - loss: 0.6488 - accuracy: 0.6415
Epoch 9/15
 - 0s - loss: 0.6487 - accuracy: 0.6405
Epoch 10/15
 - 0s - loss: 0.6486 - accuracy: 0.6415
Epoch 11/15
 - 0s - loss: 0.6488 - accuracy: 0.6412
Epoch 12/15
 - 0s - loss: 0.6488 - accuracy: 0.6412
Epoch 13/15
 - 0s - loss: 0.6489 - accuracy: 0.6405
Epoch 14/15
 - 0s - loss: 0.6488 - accuracy: 0.6412
Epoch 15/15
 - 0s - loss: 0.6493 - accuracy: 0.6408
0.6068965517241379
1    0.993103
0    0.006897
dtype: float64


So the same variables in scaled_fights is not better.

In [127]:
scaled_fights.columns

Index(['r_fighter', 'b_fighter', 'winner', 'loser', 'r_height', 'r_weight',
       'r_reach', 'r_stance', 'b_height', 'b_weight', 'b_reach', 'b_stance',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff', 'r_b_winner', 'r_orthodox', 'r_southpaw', 'r_switch',
       'r_open', 'b_orthodox', 'b_southpaw', 'b_switch', 'b_open', 'r_won'],
      dtype='object')

Try using all variable columns.

In [128]:
variable_columns = ['r_height', 'r_weight',
       'r_reach', 'r_stance', 'b_height', 'b_weight', 'b_reach', 'b_stance',
       'r_age', 'b_age', 'age_diff', 'height_diff', 'weight_diff',
       'reach_diff', 'r_b_winner', 'r_orthodox', 'r_southpaw', 'r_switch',
       'r_open', 'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']

variable_columns = ['r_height', 'b_height', 'height_diff',
                    'r_weight', 'b_weight', 'weight_diff',
                    'r_reach', 'b_reach', 'reach_diff',
                    'r_age', 'b_age', 'age_diff',
                    'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       'b_orthodox', 'b_southpaw', 'b_switch', 'b_open']

In [129]:
model = Sequential()
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn.run_analysis(model, x_columns=scaled_fights[variable_columns], y_column=scaled_fights.r_won, num_epochs=35)

Epoch 1/35
 - 0s - loss: 0.7225 - accuracy: 0.6045
Epoch 2/35
 - 0s - loss: 0.6442 - accuracy: 0.6351
Epoch 3/35
 - 0s - loss: 0.6444 - accuracy: 0.6351
Epoch 4/35
 - 0s - loss: 0.6409 - accuracy: 0.6351
Epoch 5/35
 - 0s - loss: 0.6395 - accuracy: 0.6425
Epoch 6/35
 - 0s - loss: 0.6409 - accuracy: 0.6344
Epoch 7/35
 - 0s - loss: 0.6382 - accuracy: 0.6405
Epoch 8/35
 - 0s - loss: 0.6387 - accuracy: 0.6371
Epoch 9/35
 - 0s - loss: 0.6417 - accuracy: 0.6354
Epoch 10/35
 - 0s - loss: 0.6339 - accuracy: 0.6385
Epoch 11/35
 - 0s - loss: 0.6364 - accuracy: 0.6473
Epoch 12/35
 - 0s - loss: 0.6332 - accuracy: 0.6487
Epoch 13/35
 - 0s - loss: 0.6354 - accuracy: 0.6425
Epoch 14/35
 - 0s - loss: 0.6354 - accuracy: 0.6408
Epoch 15/35
 - 0s - loss: 0.6321 - accuracy: 0.6449
Epoch 16/35
 - 0s - loss: 0.6369 - accuracy: 0.6432
Epoch 17/35
 - 0s - loss: 0.6354 - accuracy: 0.6446
Epoch 18/35
 - 0s - loss: 0.6315 - accuracy: 0.6463
Epoch 19/35
 - 0s - loss: 0.6309 - accuracy: 0.6419
Epoch 20/35
 - 0s - l

In [130]:
variable_columns = ['r_height', 'b_height', 'height_diff',
                    'r_weight', 'b_weight', 'weight_diff',
                    'r_reach', 'b_reach', 'reach_diff',
                    'r_age', 'b_age', 'age_diff',
                    #'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       #'b_orthodox', 'b_southpaw', 'b_switch', 'b_open'
                   ]

In [131]:
model = Sequential()
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn.run_analysis(model, x_columns=scaled_fights[variable_columns], y_column=scaled_fights.r_won, num_epochs=35)

Epoch 1/35
 - 0s - loss: 2.7778 - accuracy: 0.5046
Epoch 2/35
 - 0s - loss: 0.6579 - accuracy: 0.6310
Epoch 3/35
 - 0s - loss: 0.6501 - accuracy: 0.6306
Epoch 4/35
 - 0s - loss: 0.6442 - accuracy: 0.6446
Epoch 5/35
 - 0s - loss: 0.6442 - accuracy: 0.6364
Epoch 6/35
 - 0s - loss: 0.6393 - accuracy: 0.6439
Epoch 7/35
 - 0s - loss: 0.6421 - accuracy: 0.6449
Epoch 8/35
 - 0s - loss: 0.6338 - accuracy: 0.6456
Epoch 9/35
 - 0s - loss: 0.6416 - accuracy: 0.6402
Epoch 10/35
 - 0s - loss: 0.6366 - accuracy: 0.6432
Epoch 11/35
 - 0s - loss: 0.6366 - accuracy: 0.6483
Epoch 12/35
 - 0s - loss: 0.6349 - accuracy: 0.6497
Epoch 13/35
 - 0s - loss: 0.6350 - accuracy: 0.6514
Epoch 14/35
 - 0s - loss: 0.6396 - accuracy: 0.6425
Epoch 15/35
 - 0s - loss: 0.6359 - accuracy: 0.6429
Epoch 16/35
 - 0s - loss: 0.6353 - accuracy: 0.6473
Epoch 17/35
 - 0s - loss: 0.6370 - accuracy: 0.6487
Epoch 18/35
 - 0s - loss: 0.6371 - accuracy: 0.6521
Epoch 19/35
 - 0s - loss: 0.6416 - accuracy: 0.6368
Epoch 20/35
 - 0s - l

In [132]:
variable_columns = ['r_height', 'b_height', 'height_diff',
                    #'r_weight', 'b_weight', 'weight_diff',
                    'r_reach', 'b_reach', 'reach_diff',
                    'r_age', 'b_age', 'age_diff',
                    #'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       #'b_orthodox', 'b_southpaw', 'b_switch', 'b_open'
                   ]
model = Sequential()
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn.run_analysis(model, x_columns=scaled_fights[variable_columns], y_column=scaled_fights.r_won, num_epochs=35)

Epoch 1/35
 - 0s - loss: 0.8294 - accuracy: 0.5953
Epoch 2/35
 - 0s - loss: 0.6540 - accuracy: 0.6323
Epoch 3/35
 - 0s - loss: 0.6457 - accuracy: 0.6412
Epoch 4/35
 - 0s - loss: 0.6422 - accuracy: 0.6391
Epoch 5/35
 - 0s - loss: 0.6419 - accuracy: 0.6364
Epoch 6/35
 - 0s - loss: 0.6398 - accuracy: 0.6459
Epoch 7/35
 - 0s - loss: 0.6393 - accuracy: 0.6483
Epoch 8/35
 - 0s - loss: 0.6388 - accuracy: 0.6419
Epoch 9/35
 - 0s - loss: 0.6398 - accuracy: 0.6402
Epoch 10/35
 - 0s - loss: 0.6386 - accuracy: 0.6408
Epoch 11/35
 - 0s - loss: 0.6403 - accuracy: 0.6402
Epoch 12/35
 - 0s - loss: 0.6437 - accuracy: 0.6320
Epoch 13/35
 - 0s - loss: 0.6365 - accuracy: 0.6473
Epoch 14/35
 - 0s - loss: 0.6410 - accuracy: 0.6429
Epoch 15/35
 - 0s - loss: 0.6395 - accuracy: 0.6449
Epoch 16/35
 - 0s - loss: 0.6394 - accuracy: 0.6408
Epoch 17/35
 - 0s - loss: 0.6377 - accuracy: 0.6476
Epoch 18/35
 - 0s - loss: 0.6395 - accuracy: 0.6429
Epoch 19/35
 - 0s - loss: 0.6404 - accuracy: 0.6476
Epoch 20/35
 - 0s - l

In [133]:
variable_columns = ['r_height', 'b_height', 'height_diff',
                    #'r_weight', 'b_weight', 'weight_diff',
                    'r_reach', 'b_reach', 'reach_diff',
                    'r_age', 'b_age', 'age_diff',
                    #'r_orthodox', 'r_southpaw', 'r_switch', 'r_open',
       #'b_orthodox', 'b_southpaw', 'b_switch', 'b_open'
                   ]
model = Sequential()
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn.run_analysis(model, x_columns=scaled_fights[variable_columns], y_column=scaled_fights.r_won, num_epochs=35)

Epoch 1/35
 - 1s - loss: 0.6684 - accuracy: 0.6038
Epoch 2/35
 - 0s - loss: 0.6431 - accuracy: 0.6412
Epoch 3/35
 - 0s - loss: 0.6392 - accuracy: 0.6412
Epoch 4/35
 - 0s - loss: 0.6377 - accuracy: 0.6412
Epoch 5/35
 - 0s - loss: 0.6368 - accuracy: 0.6402
Epoch 6/35
 - 0s - loss: 0.6383 - accuracy: 0.6388
Epoch 7/35
 - 0s - loss: 0.6366 - accuracy: 0.6344
Epoch 8/35
 - 0s - loss: 0.6349 - accuracy: 0.6419
Epoch 9/35
 - 0s - loss: 0.6374 - accuracy: 0.6378
Epoch 10/35
 - 0s - loss: 0.6353 - accuracy: 0.6436
Epoch 11/35
 - 0s - loss: 0.6347 - accuracy: 0.6422
Epoch 12/35
 - 0s - loss: 0.6334 - accuracy: 0.6463
Epoch 13/35
 - 0s - loss: 0.6359 - accuracy: 0.6415
Epoch 14/35
 - 0s - loss: 0.6348 - accuracy: 0.6507
Epoch 15/35
 - 0s - loss: 0.6358 - accuracy: 0.6351
Epoch 16/35
 - 0s - loss: 0.6365 - accuracy: 0.6402
Epoch 17/35
 - 0s - loss: 0.6386 - accuracy: 0.6456
Epoch 18/35
 - 0s - loss: 0.6344 - accuracy: 0.6459
Epoch 19/35
 - 0s - loss: 0.6334 - accuracy: 0.6442
Epoch 20/35
 - 0s - l

In [134]:
variable_columns = ['r_height', 'b_height', 'height_diff',
                    'r_weight', 'b_weight', 'weight_diff',
                    'r_reach', 'b_reach', 'reach_diff',
                    'r_age', 'b_age', 'age_diff',
                    'r_orthodox', 'r_southpaw', 'r_switch', 'r_open', 
                    'b_orthodox', 'b_southpaw', 'b_switch', 'b_open'
                   ]
model = Sequential()
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(len(variable_columns), input_dim=len(variable_columns), activation='relu'))
model.add(Dense(2, activation='softmax'))
#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
dnn.run_analysis(model, x_columns=scaled_fights[variable_columns], y_column=scaled_fights.r_won, num_epochs=35)

Epoch 1/35
 - 1s - loss: 0.6452 - accuracy: 0.6412
Epoch 2/35
 - 0s - loss: 0.6407 - accuracy: 0.6412
Epoch 3/35
 - 0s - loss: 0.6394 - accuracy: 0.6497
Epoch 4/35
 - 0s - loss: 0.6395 - accuracy: 0.6361
Epoch 5/35
 - 0s - loss: 0.6360 - accuracy: 0.6412
Epoch 6/35
 - 0s - loss: 0.6379 - accuracy: 0.6429
Epoch 7/35
 - 0s - loss: 0.6355 - accuracy: 0.6466
Epoch 8/35
 - 0s - loss: 0.6331 - accuracy: 0.6497
Epoch 9/35
 - 0s - loss: 0.6345 - accuracy: 0.6456
Epoch 10/35
 - 0s - loss: 0.6345 - accuracy: 0.6456
Epoch 11/35
 - 0s - loss: 0.6350 - accuracy: 0.6442
Epoch 12/35
 - 0s - loss: 0.6350 - accuracy: 0.6405
Epoch 13/35
 - 0s - loss: 0.6333 - accuracy: 0.6429
Epoch 14/35
 - 0s - loss: 0.6332 - accuracy: 0.6483
Epoch 15/35
 - 0s - loss: 0.6315 - accuracy: 0.6456
Epoch 16/35
 - 0s - loss: 0.6344 - accuracy: 0.6500
Epoch 17/35
 - 0s - loss: 0.6334 - accuracy: 0.6432
Epoch 18/35
 - 0s - loss: 0.6326 - accuracy: 0.6504
Epoch 19/35
 - 0s - loss: 0.6309 - accuracy: 0.6483
Epoch 20/35
 - 0s - l

So it does not do a good job of predicting based on physical characteristics.  Maybe I can find a way to use their record of past fights.

In [135]:
def get_fights_for_fighter(fighter_name, fights_df = fights):
    return fights_df[(fights_df.r_fighter == fighter_name) | (fights_df.b_fighter == fighter_name)]

In [136]:
class RecordSummary:
    def __init__(self, name, wins, losses, ties):
        self.name = name
        self.wins = wins
        self.losses = losses
        self.ties = ties
        
    def __str__(self):
        return '{ name: ' + self.name + ', ' + str(self.wins) + ', ' + str(self.losses) + ', ' + str(self.ties) + ', ' + str(self.wins / (self.wins + self.losses)) + '}'
    
    def __repr__(self):
        return self.__str__()
    
    def __lt__(self, other):
        return (self.wins < other.wins)

In [137]:
def get_record_summary(fighter_name, fights_df = fights):
    #record = get_record(fighter_name, fights_df)
    summary = {}
    summary['wins'] = (fights_df['winner'] == fighter_name).sum()
    summary['losses'] = (fights_df['winner'] == fighter_name).sum()
    summary['losses'] = (fights_df['loser'] == fighter_name).sum()
    summary['draws'] = (((fights_df['r_fighter'] == fighter_name) | (fights_df['b_fighter'] == fighter_name) ) & (fights_df['winner'] == 'None')).sum()
    summary['win loss ratio'] = summary ['wins'] / (summary['wins'] + summary['losses'])
    
    return summary

In [138]:
all_fighters = set(fights.r_fighter).union(set(fights.b_fighter))
len(all_fighters)

1423

In [139]:
all_summaries = []
for fighter in all_fighters:
    #print(fighter, ':  ', get_record_summary(fighter))
    m = get_record_summary(fighter)
    rs = RecordSummary(fighter, m['wins'], m['losses'], m['draws'])
    all_summaries.append(rs)

all_summaries.sort()
all_summaries = all_summaries[::-1]
all_summaries[:20]

C:\Users\paulj_1e1uzlz\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in longlong_scalars
  


[{ name: Donald Cerrone, 23, 9, 0, 0.71875},
 { name: Demian Maia, 20, 9, 0, 0.6896551724137931},
 { name: Georges St-Pierre, 20, 2, 0, 0.9090909090909091},
 { name: Michael Bisping, 19, 9, 0, 0.6785714285714286},
 { name: Rafael Dos Anjos, 18, 9, 0, 0.6666666666666666},
 { name: Jim Miller, 17, 12, 1, 0.5862068965517241},
 { name: Jon Jones, 17, 1, 1, 0.9444444444444444},
 { name: Anderson Silva, 17, 6, 1, 0.7391304347826086},
 { name: Diego Sanchez, 17, 11, 0, 0.6071428571428571},
 { name: Dustin Poirier, 17, 4, 1, 0.8095238095238095},
 { name: Frankie Edgar, 16, 6, 1, 0.7272727272727273},
 { name: Max Holloway, 16, 4, 0, 0.8},
 { name: Ryan Bader, 15, 5, 0, 0.75},
 { name: Demetrious Johnson, 15, 2, 1, 0.8823529411764706},
 { name: Junior Dos Santos, 15, 4, 0, 0.7894736842105263},
 { name: Tony Ferguson, 15, 1, 0, 0.9375},
 { name: Lyoto Machida, 15, 8, 0, 0.6521739130434783},
 { name: Gleison Tibau, 15, 12, 0, 0.5555555555555556},
 { name: Nate Diaz, 14, 9, 0, 0.6086956521739131},


So for fights and scaled_fights, add a column for x_previous_wins, x_previous_losses, x_previous_ties.

We need to add the date of the fight back in.

In [ ]:
'date' in fights.columns